### Transfer learning

Źródło: https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975

Dane: https://www.microsoft.com/en-us/download/details.aspx?id=54765

In [1]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')

Using TensorFlow backend.


574717952/574710816 [==============================] - 307s 1us/step

In [3]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [25]:
h,w = 150, 150

In [75]:
from keras.applications.vgg19 import VGG19

h,w = 150, 150
model = VGG19(weights='imagenet',include_top=False,input_shape=(h,w,3))

In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

In [22]:
?VGG19

In [21]:
from keras.models import Model

In [36]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [37]:
new_model = Sequential()
new_model.add(model)
new_model.add(top_model)
base_model = new_model

In [61]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
sequential_5 (Sequential)    (None, 1)                 2097665   
Total params: 22,122,049
Trainable params: 22,122,049
Non-trainable params: 0
_________________________________________________________________


In [64]:
base_model.layers[0].trainable = False

In [65]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
sequential_5 (Sequential)    (None, 1)                 2097665   
Total params: 22,122,049
Trainable params: 2,097,665
Non-trainable params: 20,024,384
_________________________________________________________________


In [67]:
from keras import applications

from keras.preprocessing.image import ImageDataGenerator

from keras import optimizers

In [69]:
# dimensions of our images.
img_width, img_height = 100, 100

train_data_dir = '/home/kodolamacz/Pobrane/kagglecatsanddogs_3367a/PetImages/data/train/'
validation_data_dir = '/home/kodolamacz/Pobrane/kagglecatsanddogs_3367a/PetImages/data/validation/'
nb_train_samples = 1607
nb_validation_samples = 363
epochs = 50
batch_size = 16

# build the VGG19 network
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(100, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# add the model on top of the convolutional base
model = Sequential()
model.add(base_model)
model.add(top_model)
print(model.summary())
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)

model.layers[0].trainable = False
model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),metrics=['accuracy'])


# prepare data augmentation configuration
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,target_size=(img_height, img_width), 
                                                        batch_size=batch_size,class_mode='binary')

model.fit_generator(train_generator, samples_per_epoch=nb_train_samples, epochs=epochs, 
                    validation_data=validation_generator, nb_val_samples=nb_validation_samples)

Model loaded.
Found 1608 images belonging to 2 classes.
Found 363 images belonging to 2 classes.


/home/kodolamacz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:92: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/kodolamacz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:92: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=50, validation_data=<keras_pre..., steps_per_epoch=100, validation_steps=363)`


Epoch 1/50
 30/100 [========>.....................] - ETA: 2:57 - loss: 0.7643 - acc: 0.5208

StopIteration: cannot identify image file '/home/kodolamacz/Pobrane/kagglecatsanddogs_3367a/PetImages/data/train/cats/666.jpg'

In [71]:
# dimensions of our images.
img_width, img_height = 100, 100

train_data_dir = '/home/kodolamacz/Pobrane/kagglecatsanddogs_3367a/PetImages/data/train/'
validation_data_dir = '/home/kodolamacz/Pobrane/kagglecatsanddogs_3367a/PetImages/data/validation/'
nb_train_samples = 1607
nb_validation_samples = 363
epochs = 50
batch_size = 16

# build a classifier model to put on top of the convolutional model

top_model = Sequential()
top_model.add(Flatten(input_shape=(h,w,3)))
top_model.add(Dense(1000, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(100, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),metrics=['accuracy'])

# prepare data augmentation configuration

train_datagen = ImageDataGenerator(rescale=1./255)#, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), 
                                                    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,target_size=(img_height, img_width), 
                                                        batch_size=batch_size,class_mode='binary')



# fine-tune the model

model.fit_generator(train_generator, samples_per_epoch=nb_train_samples, epochs=epochs, 
                    validation_data=validation_generator, nb_val_samples=nb_validation_samples)

Found 1607 images belonging to 2 classes.
Found 363 images belonging to 2 classes.


/home/kodolamacz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:72: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/kodolamacz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:72: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=50, validation_data=<keras_pre..., steps_per_epoch=100, validation_steps=363)`


Epoch 1/50
100/100 [==============================] - 293s 3s/step - loss: 0.6080 - acc: 0.6656 - val_loss: 0.5592 - val_acc: 0.7107

KeyboardInterrupt: 

## Ucinanie sieci gdzieś w środku

In [96]:
input_new = model.input

In [104]:
vgg_features = model.layers[6].output

In [ ]:
d = Dense(1000)(vgg_features) 
output = Dense(1,activation="sigmoid")(d) 

In [102]:
model_new = Model(input_new, output)

In [103]:
model_new.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
Total para